In [57]:
import pyedflib
import numpy as np

from scipy import signal
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools as iter
import math

In [6]:
#read dataset
csv_file = 'mros-visit1-dataset-0.3.0.csv'
df = pd.read_csv(csv_file, dtype={"nsrrid": str, "poordi4": int}, low_memory=False)
AHI = df[['nsrrid', 'poordi4']] #select only two columns

#select only 50 patients from each group (normal/abnormal)
n_data = 100
group1 = AHI[AHI.poordi4 < 5][0:n_data/2] #group1 : normal
group1['class'] = 0
group2 = AHI[AHI.poordi4 >= 30][0:n_data/2] #group2 : OSA patient
group2['class'] = 1
all_data = pd.concat([group1, group2])

In [13]:
# notch function applied from what using in SSVEP
def notch_filter(ecg):
    nyq = 0.5 * sampling_rate
    low = 60 / nyq
    high = 61 / nyq
    order = 2
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.lfilter(b, a, ecg)

In [46]:
## get ECGs ##

#set params
sampling_rate = 512 #number of samplings per second
second_hour_start = sampling_rate * 60 * 60
fifth_hour_end = sampling_rate * 60 * 60 * 5
length = fifth_hour_end-second_hour_start
ECGs = np.zeros(shape=(n_data, length + 1))

for i, it in enumerate(iter.izip(all_data['nsrrid'], all_data['class'])):
    try:
        #one loop per subject
        print 'round', i
        f_name = "../mros/polysomnography/edfs/visit1/mros-visit1-" + it[0].lower() + ".edf"
        f = pyedflib.EdfReader(f_name)

        '''
        #Uncomment to see shape of data
        if i == 0:
            n = f.signals_in_file
            signal_labels = f.getSignalLabels()
            print "This file includes", n, "signals :"
            print "(list of signal : number of samples in each channel)"
            for i, s in enumerate(signal_labels):
                print '\t', s, ':', f.getNSamples()[i]
        '''

        #select only ECG
        ecg_L = f.readSignal(9)

        #select only data from 2nd to 5th hours
        ecg_L_4hrs = ecg_L[second_hour_start : fifth_hour_end]
  
        #bandpass filter
        filtered_signal = notch_filter(ecg_L_4hrs)
        
        #add all patients' ECGs and class to numpy array
        ECGs[i] = np.append(filtered_signal, it[1])
        
        '''
        #write selected period of signal to file
        fw_name = "./ECGs/visit1/mros-visit1-" + it[0].lower() + "-4hours.edf"
        np.save(fw_name, np.array(filtered_signal))
        '''
        
        print "Select ECG data only from 2nd to 5th hours =", length, filtered_signal.shape, "samplings from all", len(ecg_L), "samplings."
    
    except Exception, e:
        print 'FAIL at', f_name, e
    finally:
        f._close()
    
print ECGs.shape

round 0
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 16128000 samplings.
round 1
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 22256640 samplings.
round 2
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 19046400 samplings.
round 3
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 20259840 samplings.
round 4
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 16204800 samplings.
round 5
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 22195200 samplings.
round 6
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 19184640 samplings.
round 7
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 22256640 samplings.
round 8
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 23040000 samplings.
r

Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 17617920 samplings.
round 75
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 17971200 samplings.
round 76
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 20044800 samplings.
round 77
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 32271360 samplings.
round 78
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 19722240 samplings.
round 79
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 19691520 samplings.
round 80
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 17817600 samplings.
round 81
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 22609920 samplings.
round 82
Select ECG data only from 2nd to 5th hours = 7372800 (7372800,) samplings from all 22256640 samplings.
r

In [63]:
## Extract ECG of apnea period
# specify Low (normal) and High (abnormal) OSA severity patients
patient_id_low = 'aa0014'
patient_id_high = 'aa0029'

# get ECG from edf file
f1_name = "../mros/polysomnography/edfs/visit1/mros-visit1-" + patient_id_low + ".edf"
f2_name = "../mros/polysomnography/edfs/visit1/mros-visit1-" + patient_id_high + ".edf"
try:
    f1 = pyedflib.EdfReader(f1_name)
    f2 = pyedflib.EdfReader(f2_name)

    #select only ECG
    ecg_L_p1 = f1.readSignal(9)
    ecg_L_p2 = f2.readSignal(9)
    
except Exception, e:
    print 'Cannot read EDF file.', e
finally:
    f1._close()
    f2._close()
    
# apnea typically lasts 20 to 40 seconds (can up to more than 1 min)
# try to select events which occur in comparable duration
start1 = 24241.6
start2 = 35564.2
duration = 40

hypopnea_event_p1_start = sampling_rate * int(start1) #duration = 40.1
hypopnea_event_p2_start = sampling_rate * int(start2) #duration = 40.5
hypopnea_event_p1_end = sampling_rate * int((start1 + duration))
hypopnea_event_p2_end = sampling_rate * int((start2 + duration))

print "Getting ECG from normal subject #", patient_id_low
print "Start:", hypopnea_event_p1_start, "End:", hypopnea_event_p1_end
print "Getting ECG from OSA subject #", patient_id_high
print "Start:", hypopnea_event_p2_start, "End:", hypopnea_event_p2_end
ecg_L_p1 = ecg_L_p1[hypopnea_event_p1_start: hypopnea_event_p1_end]
ecg_L_p2 = ecg_L_p2[hypopnea_event_p2_start: hypopnea_event_p2_end]

print "Standard Deviation of Normal subject:", np.std(ecg_L_p1)
print "Standard Deviation of OSA patient:", np.std(ecg_L_p2)

Getting ECG from normal subject # aa0014
Start: 12411392 End: 12431872
Getting ECG from OSA subject # aa0029
Start: 18208768 End: 18229248
Standard Deviation of Normal subject: 0.0533151377440947
Standard Deviation of OSA patient: 0.16232546513390486


In [69]:
def get_ecg(pid, start, duration, subject_type = 'normal'):
    # get ECG from edf file
    f_name = "../mros/polysomnography/edfs/visit1/mros-visit1-" + pid.lower() + ".edf"
    try:
        f = pyedflib.EdfReader(f_name)

        #select only ECG
        ecg_L = f.readSignal(9)

    except Exception, e:
        print 'Cannot read EDF file.', e
    finally:
        f._close()

    # apnea typically lasts 20 to 40 seconds (can up to more than 1 min)
    # try to select events which occur in comparable duration
    hypopnea_event_p_start = sampling_rate * int(start1)
    hypopnea_event_p_end = sampling_rate * int((start1 + duration))

    print "Getting ECG from subject #", pid, "(", subject_type.upper() , ")"
    print "Start:", hypopnea_event_p_start, "End:", hypopnea_event_p_end
    
    ecg_L = ecg_L[hypopnea_event_p_start: hypopnea_event_p_end]

    return ecg_L
    

In [70]:
pid = 'aa0029'
start = 35564.2
duration = 40
subject_type = 'osa'

ecg_L = get_ecg(pid, start, duration, subject_type)
print "Standard Deviation:", np.std(ecg_L)

Getting ECG from subject # aa0029 ( OSA )
Start: 12411392 End: 12431872
Standard Deviation: 0.1664381788512328
